### This jupyter notebook walks you through how to build a model in Python and deploy it in SAS Intelligent Decisioning

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\sinnsr\OneDrive - SAS\FLXathon 2020\gitlab\alpha-bravo-charlie\datasets\python_xsell_data.csv')

Make sure to put the _r'_ in the above pd.read_csv() function. While scoring in ID, it works because the execution is in CAS. But while publishing, the execution is in MAS that behave a bit differently.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np

In [4]:
X=df[["Claim_Amt","Current_Premium","Car_Price","Car_Age","Rating_Category"]]

In [5]:
X.head()

,Claim_Amt,Current_Premium,Car_Price,Car_Age,Rating_Category
0,1300,70053.0,68400.0,7.0,NaN
1,1700,97800.0,112000.0,3.0,NaN
2,1700,30548.0,40320.0,9.0,37.113614
3,2000,22608.0,NaN,18.0,NaN
4,2000,45000.0,55000.0,3.0,NaN


In [6]:
colmmn_list = ['Claim_Amt','Current_Premium','Car_Price','Car_Age','Rating_Category']

In [7]:
import sklearn
print(sklearn.__version__)

0.22


In [8]:
from sklearn_pandas import DataFrameMapper

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [10]:
RFclassifier = RandomForestRegressor(n_estimators = 100, max_depth=2, random_state=54321)

#### Fit the model

In [11]:
pipeline_reg = Pipeline([
     #Step1 - normalize data
    ('mapper',  DataFrameMapper([
                  (["Claim_Amt","Current_Premium","Car_Price","Car_Age","Rating_Category"], [SimpleImputer(missing_values = np.nan , strategy = 'median')])
    ])),   
    
    ('classifier',RFclassifier) #step2 - classifier
])

In [12]:
pipeline_reg.fit(X,df["Offer_Score"])

Pipeline(memory=None,
         steps=[('mapper',
                 DataFrameMapper(default=False, df_out=False,
                                 features=[(['Claim_Amt', 'Current_Premium',
                                             'Car_Price', 'Car_Age',
                                             'Rating_Category'],
                                            [SimpleImputer(add_indicator=False,
                                                           copy=True,
                                                           fill_value=None,
                                                           missing_values=nan,
                                                           strategy='median',
                                                           verbose=0)])],
                                 input_df=False, sparse=False)),
                ('classifier',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse',

#### Export the Model pickle file. Copy this file to a server location which SAS ID can access 

In [35]:
import pickle 
pklfile = open("RForest_XSELLreg_obj.pkl",'wb')
pickle.dump(pipeline_reg, pklfile)
pklfile.close()

### Go to SAS Intelligent Decisioning. Make a sample Decision. Add a python code file in SAS ID and copy the below code inside it.

In [ ]:
''' List all output parameters as comma-separated values in the "Output:" docString. Do not specify "None" if there is no output parameter. '''

import pandas as pd
import pickle

colmmn_list = ['Claim_Amt','Current_Premium','Car_Price','Car_Age','Rating_Category']

pickle_location = '/home/sasdemo/RForest_XSELLreg_obj.pkl'

loaded_model = pickle.load(open(pickle_location, 'rb'))


def execute (Claim_Amt, Current_Premium, Car_Price, Car_Age, Rating_Category):
   'Output: result'
   df = pd.DataFrame([[Claim_Amt, Current_Premium, Car_Price, Car_Age, Rating_Category]], columns=colmmn_list)
   result1 = loaded_model.predict(df)
   result = float(round(result1[0],4))
   #result = 5
   return result


##### The above code will run the predict funciton using the pkl file and operationlize your model in a governed environment like SAS ID